In [22]:
# Create pandas DataFrame
# df = pd.DataFrame( {"A": [7001, 8001, 9001]} )

In [1]:
import csv
import pandas as pd
import numpy as np

In [2]:
# Importing the table with Panama connections

all_edges = pd.read_csv("ICIJ Data/all_edges.csv", encoding='ISO-8859-1')
print(len(all_edges))
print(all_edges.head())

1265690
     node_1         rel_type    node_2
0  11000001  intermediary_of  10208879
1  11000001  intermediary_of  10198662
2  11000001  intermediary_of  10159927
3  11000001  intermediary_of  10165779
4  11000001  intermediary_of  10152967


In [3]:
# Importing the perfect dictionary

perfect = pd.read_csv("Merged/perfect_dictionary.csv", encoding='ISO-8859-1')
print(len(perfect))
print(perfect.head())

3677
          Country      Bloomberg Name    Original Panama Name   Node ID  \
0  United Kingdom  olafur gudmundsson  MR. OLAFUR GUDMUNDSSON  12190271   
1  United Kingdom  olafur gudmundsson   Olafur S. Gudmundsson  12190499   
2  United Kingdom        wang xiaodan            WANG Xiaodan  12193848   
3  United Kingdom       mario cordoni       Mr. Mario CORDONI  12194229   
4  United Kingdom       mario cordoni       Mr. Mario CORDONI  12199491   

                            ICIJ ID  
0  CAFD16E4BA76071D4DCC7C218EA14F13  
1  8A920B695E492F44B19DB4FC9B31415D  
2  33747075E3C9B41F040631878A8CA36A  
3  C7368607523BC2F37C6D57E3495CCCDF  
4  10E9D4BAB8BBB0C9A103B570C029B459  


In [4]:
# Creating a table with only officers

officer_edges = all_edges[all_edges['rel_type'] == 'officer_of']
del officer_edges['rel_type']
print(len(officer_edges))
print(officer_edges.head())

581476
          node_1    node_2
319121  12000001  10073324
319122  12000002  10148386
319123  12000003  10024966
319124  12000004  10004763
319125  12000005  10206741


In [ ]:
# Creating a dictionary with officers' IDs as keys and ENTITIES as values

edges_dict = {k: g["node_2"].tolist() for k,g in officer_edges.groupby("node_1")}
print(len(edges_dict))
#print(edges_dict)

In [130]:
# Creating a table with only proper OFFICER-ENTITY relationships

good_edges_officers = pd.DataFrame({'node_1': perfect[perfect['Node ID'].isin(edges_dict.keys())]['Node ID']})
good_edges = pd.merge(good_edges_officers, officer_edges, how='left')
good_edges.columns = ['Officer', 'Entity']
print(len(good_edges))
print(good_edges.head())

# Exporting the table with proper OFFICER-ENTITY relationships

good_edges.to_csv('Merged/officer_entity_ids.csv')

4483
    Officer    Entity
0  12190271  10145953
1  12190499  10134528
2  12193848  10022514
3  12194229  10033002
4  12194229  10034641


In [148]:
# Creating a table with entities and names

good_edges.columns = ['Node ID', 'Entity']
ent_name = pd.merge(good_edges, perfect, how='left')
print(len(ent_name))
print(ent_name.head())

4929
    Node ID    Entity         Country      Bloomberg Name  \
0  12190271  10145953  United Kingdom  olafur gudmundsson   
1  12190499  10134528  United Kingdom  olafur gudmundsson   
2  12193848  10022514  United Kingdom        wang xiaodan   
3  12194229  10033002  United Kingdom       mario cordoni   
4  12194229  10034641  United Kingdom       mario cordoni   

     Original Panama Name                           ICIJ ID  
0  MR. OLAFUR GUDMUNDSSON  CAFD16E4BA76071D4DCC7C218EA14F13  
1   Olafur S. Gudmundsson  8A920B695E492F44B19DB4FC9B31415D  
2            WANG Xiaodan  33747075E3C9B41F040631878A8CA36A  
3       Mr. Mario CORDONI  C7368607523BC2F37C6D57E3495CCCDF  
4       Mr. Mario CORDONI  C7368607523BC2F37C6D57E3495CCCDF  
3328


In [149]:
# Importing rock_solid_truth

truth = pd.read_csv("Merged/rock_solid_truth.csv", encoding='ISO-8859-1')
print(len(truth))
print(truth.head())

6240
   _          Panama Name         Bloomberg Name Panama Country  \
0  1  lee shing bun benny    benny shing bun lee      Hong Kong   
1  2  thomas lee kang bor  lee kang bor "thomas"      Hong Kong   
2  2  thomas lee kang bor  lee kang bor "thomas"      Hong Kong   
3  2  thomas lee kang bor  lee kang bor "thomas"      Hong Kong   
4  2  thomas lee kang bor  lee kang bor "thomas"      Hong Kong   

                            Cname           Cname Short  Position Chairman?  \
0        Asia Payment Systems Inc  Asia Payment Systems  Former P       NaN   
1       Thomas Lee & Partners Ltd  Thomas Lee & Partner       NaN         Y   
2  Fittec International Group Ltd  Fittec International       NaN       NaN   
3           Cig Yangtze Ports PLC  Cig Yangtze Ports PL       NaN       NaN   
4          Sparkle Roll Group Ltd  Sparkle Roll Group L       NaN       NaN   

  Board Member?   Domicile    Ticker Type          ISIN  
0             Y        NaN       NaN  NaN           NaN  
1

In [153]:
# Creating a dictionary and a dataframe with NEW number IDs of companies as keys and companies names as values

names_list = sorted(list(set(truth["Cname Short"].str.upper())))
names_ids = [x for x in range(0, len(names_list))]
names_pandas = pd.DataFrame(index = range(len(names_list)), columns = ['Company Name'])
names_pandas['Company Name'] = pd.DataFrame(names_list)
names_pandas['Company ID'] = pd.DataFrame(names_ids)



comps_dict = {}
for i in range(len(names_list)):
    comps_dict[names_list[i]] = i

print(len(names_pandas))    
print(names_pandas.head())

# Exporting the dataframe to csv

names_pandas.to_csv('Merged/real_companies_dict.csv')

# Saving the dictionary 

np.save('real_companies_dict.npy', comps_dict) 

5554
           Company Name  Company ID
0    (CA)SUPERIOR COURT           0
1     (CO)SUPREME COURT           1
2  (IDN)MINISTRY OF FIN           2
3  (IDN)OFFICE OF THE P           3
4  (ISR)OFFICE OF PRIME           4


In [164]:
# Creating a dictionary with entities as keys and companies IDs as values - FOR HIDDEN CONNECTIONS

truth_part = truth[['Bloomberg Name', 'Cname Short']]
truth_part['Cname Short'] = truth_part['Cname Short'].str.upper()
truth_part.columns = ['Bloomberg Name', 'Company Name']
truth_part_new = pd.merge(truth_part, names_pandas, how='left')
entity_comps = pd.merge(ent_name, truth_part_new, how='inner')
entity_comps = entity_comps[['Entity', 'Company ID']]
print(len(entity_comps))
print(entity_comps.head())

company_dict2 = {k: g["Company ID"].tolist() for k,g in entity_comps.groupby("Entity")}
print(len(company_dict2))

# Saving the dictionary 

np.save('companies_offshores_dict.npy', company_dict2)

//anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app


16052
     Entity  Company ID
0  10145953         210
1  10134528         210
2  10022514        1077
3  10033002        1309
4  10034641        1309
3311


In [195]:
# Creating a list of direct connections

names_comps1 = truth_part_new[['Bloomberg Name', 'Company ID']]
names_comps2 = names_comps1.copy()
names_comps2.columns = ['Bloomberg Name', 'Company ID 2']
direct = pd.merge(names_comps1, names_comps2, how='inner')
del direct['Bloomberg Name']
direct = direct[~direct.duplicated()]
direct = direct[direct['Company ID'] != direct['Company ID 2']]
print(len(direct))
print(direct.head())

48676
   Company ID  Company ID 2
2        5025          1874
3        5025          1137
4        5025          4753
5        1874          5025
7        1874          1137


In [194]:
# Creating a list of hidden+direct connections

entity_comps1 = entity_comps.copy()
entity_comps2 = entity_comps1.copy()
entity_comps2.columns = ['Entity', 'Company ID 2']
hidden_direct = pd.merge(entity_comps1, entity_comps2, how='inner')
del hidden_direct['Entity']
hidden_direct = hidden_direct[hidden_direct['Company ID'] != hidden_direct['Company ID 2']]
hidden_direct = hidden_direct[~hidden_direct.duplicated()]
print(len(hidden_direct))
print(hidden_direct.head())

67142
    Company ID  Company ID 2
13        4822          1773
14        4822            23
19        4822          3465
20        4822          4353
21        4822          3577


In [201]:
# Creating a list of hidden connections

hidden_direct_copy = hidden_direct.copy()
direct_copy = direct.copy()
hidden_direct_copy['key'] = hidden_direct_copy['Company ID'].astype(str) + hidden_direct_copy['Company ID 2'].astype(str)
direct_copy['key'] = direct_copy['Company ID'].astype(str) + direct_copy['Company ID 2'].astype(str)
hidden = hidden_direct_copy[~hidden_direct_copy['key'].isin(direct_copy['key'])]
del hidden['key']
print(len(hidden))
print(hidden.head())

19492
    Company ID  Company ID 2
19        4822          3465
20        4822          4353
21        4822          3577
47        1773          3465
48        1773          4353


In [206]:
# Sorting and exporting to csv

hidden.sort(['Company ID','Company ID 2'], ascending=True)
direct.sort(['Company ID','Company ID 2'], ascending=True)

hidden.to_csv('Merged/hidden_connections.csv')
direct.to_csv('Merged/direct_connections.csv')

//anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:3: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)
  app.launch_new_instance()
//anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:4: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)
